In [1]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns',None)

In [2]:
# загрузка данных
df_2017 = pd.read_excel('mixes_2017_b1.xlsx')
df_2018 = pd.read_excel('mixes_2018_b1.xlsx')
df_2019 = pd.read_excel('mixes_2019_b1.xlsx')
df_2020 = pd.read_excel('mixes_2020_b1.xlsx')
df_2021 = pd.read_excel('mixes_2021_b1.xlsx')
df_prop = pd.read_excel('properties.xlsx')

In [3]:
def prepare_and_concat(df_2017,df_2018,df_2019,df_2020,df_2021,df_prop):
    # rename columns df_prop
    df_prop.rename(columns={'Год' : 'year',
                        'Концентрат':'Conc',
                        'σ':'sigma',
                        'КТЦ эксп.':'KTC'},inplace=True)
    
    # 2019год: два столбца 31 и 31.1 объединим в один
    df_2019[['Производитель 31','Производитель 31.1']] = df_2019[['Производитель 31','Производитель 31.1']].fillna(0)
    df_2019['Производитель 31'] += df_2019['Производитель 31.1']
    df_2019.drop(columns=['Производитель 31.1'],inplace=True)
    
    # 2021 год: Удалим Столбец1
    df_2021 = df_2021.drop(columns=['Столбец1'])
    
    # собираем общий dataframe
    df_mixes = pd.concat([df_2017,df_2018,df_2019,df_2020,df_2021])
    
    #Объединим  Производитель 1 и Производитель 2 
    df_mixes[['Производитель 1 ','Производитель 2']] = df_mixes[['Производитель 1 ','Производитель 2']].fillna(0)
    df_mixes['Производитель 2'] += df_mixes['Производитель 1 ']
    df_mixes.drop(columns=['Производитель 1 '],inplace=True)
    
    #Добавим  столбец год
    df_mixes['year']=df_mixes['Дата'].dt.year

    #Избавимся от пропусков в дате(==2смена)
    df_mixes = df_mixes[~df_mixes['year'].isnull()] 
    df_mixes['year'] = df_mixes['year'].astype('int64')
    
    #Удалить даты "13-08-2019", "14-08-2019", "17-11-2019"
    df_mixes = df_mixes.loc[~df_mixes['Дата'].isin(['2019-08-13','2019-08-14','2019-11-17'])].reset_index(drop=True)
    
    return df_mixes

df_mixes = prepare_and_concat(df_2017,df_2018,df_2019,df_2020,df_2021,df_prop)
df_mixes.head()

,Дата,"Состав шихты,%(спек.)","Состав шихты,%(в т.ч. Марка Ж)","Состав шихты,%(отощ.)","Состав шихты,%(в т.ч. кокс.)",Производитель 2,Производитель 3,Производитель 4,Производитель 5,Производитель 6,Производитель 7,Производитель 8,Производитель 9,Производитель 10,Производитель 11,Производитель 12,Производитель 13,Производитель 14,Производитель 15,Производитель 16,Производитель 17,Производитель 18,Производитель 19,Производитель 20,Производитель 21,Производитель 22,Производитель 23,Производитель 24,Производитель 25,Производитель 26,Производитель 27,Производитель 28,Производитель 29,Производитель 30,Производитель 31,Производитель 32,Производитель 33,Производитель 34,Производитель 35,Производитель 36,Производитель 37,Производитель 38,Производитель 39,Производитель 40,Производитель 41,Производитель 42,Производитель 43,Производитель 44,Производитель 45,"Качество шихты (помол,%)","Качество шихты (пыль,%)","Качество шихты (Технический анализ, % — влага)","Качество шихты (Технический анализ, % — зола)","Качество шихты (Технический анализ, % — летуч.)","Качество шихты (Мд. Серы,%)",Качество шихты (пласт. слой мм),Расчетн расх. Коэф угл/кокс т/т,"Качество кокса,% (Технический анализ,% — зола)","Качество кокса,% (Технический анализ,% — летуч.)","Качество кокса,% (Технический анализ,% — сера)","Качество кокса,% (Показатели прочности,% — M25)","Качество кокса,% (Показатели прочности,% — M10)","Качество кокса,% (Показатели прочности,% — CSR)","Качество кокса,% (Показатели прочности,% — CRI)","Качество кокса,% (Гранулометрический состав,% — ""+80"")","Качество кокса,% (Гранулометрический состав,% — ""80-60"")","Качество кокса,% (Гранулометрический состав,% — ""60-40"")","Качество кокса,% (Гранулометрический состав,% — ""40-25"")","Качество кокса,% (Гранулометрический состав,% — ""25-0"")","Средн. Размер куска кокса,мм","Оборот печей, час",Производитель 46,"Качество кокса,% (Технический анализ,% — влага)",Производитель 47,Производитель 48,Производитель 49,Производитель 50,Производитель 51,Производитель 52,Производитель 53,Производитель 54,Производитель 55,Производитель 56,Производитель 57,Производитель 58,year
0,2017-01-01,50.0,29.8,50.0,28.0,33.0,NaN,NaN,5.0,NaN,NaN,10.0,NaN,10.0,6.0,NaN,NaN,12.0,NaN,NaN,20.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.9,39.9,8.4,8.8,26.1,0.63,17.0,1.324987,11.3,1.1,0.61,88.3,7.8,NaN,NaN,14.5,40.1,32.8,9.4,3.2,60.9750,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
1,2017-01-02,50.0,29.8,50.0,28.0,33.0,NaN,NaN,5.0,NaN,NaN,10.0,NaN,10.0,6.0,NaN,NaN,12.0,NaN,NaN,20.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,40.1,8.5,8.5,25.2,0.62,16.0,1.315316,11.0,1.0,0.59,88.5,7.4,60.9,28.4,5.3,34.7,44.0,13.1,2.9,55.6800,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
2,2017-01-03,50.0,29.8,50.0,28.0,33.0,NaN,NaN,5.0,NaN,NaN,10.0,NaN,10.0,6.0,NaN,NaN,12.0,NaN,NaN,20.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.6,39.1,9.2,8.1,25.7,0.63,15.0,1.320671,10.9,1.2,0.59,88.3,7.7,NaN,NaN,11.2,47.5,32.8,6.1,2.4,62.0125,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
3,2017-01-04,50.0,29.8,50.0,28.0,33.0,NaN,NaN,5.0,NaN,NaN,10.0,NaN,10.0,6.0,NaN,NaN,12.0,NaN,NaN,20.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.1,38.7,9.2,8.2,25.7,0.63,15.0,1.320671,11.3,1.2,0.60,88.2,7.7,NaN,NaN,20.3,44.8,25.9,6.2,2.8,64.9450,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
4,2017-01-05,50.0,29.8,50.0,28.0,33.0,NaN,NaN,5.0,NaN,NaN,10.0,NaN,10.0,6.0,NaN,NaN,12.0,NaN,NaN,20.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.6,39.0,8.2,8.6,25.1,0.62,17.0,1.314250,11.2,1.1,0.60,88.7,7.7,NaN,NaN,11.5,44.8,32.2,7.6,3.9,60.7675,19.5,NaN,NaN,N

In [4]:
def mixes_prop(df_mixes,df_prop):
    
    cols = ['R0', 'sigma', 'Vt', 'I', 'Io', 'SI', 'KTC', 'MF',
       'MF_spec', 'MF_otosh', 'CSR_carb',]
    idx = ['year', 'Conc']
    
    df_mixes = df_mixes.assign(group=lambda d: d.groupby('year').cumcount())
    a=df_prop.merge(df_mixes.melt(id_vars=['year', 'group'], var_name='Conc'), 
           on=idx).set_index(idx+['group']).reset_index().pivot_table(index=['year', 'group'], columns='Conc', fill_value=0)
    
    a.columns = a.columns.map('_'.join)
    data_mixes = (df_mixes.merge(a, left_on=['year', 'group'], right_index=True).drop(columns='group'))
    data_mixes = data_mixes.reindex(columns=sorted(data_mixes.columns.tolist()))
    
    # Заполним пропуски в столбцах Производительность {i}
    for i in range(1,60):
        if f'Производитель {i}' in data_mixes.columns:
            data_mixes[f'Производитель {i}'] = data_mixes[f'Производитель {i}'].fillna(0)
        else:
            continue
        
    return data_mixes

data_mixes = mixes_prop(df_mixes,df_prop)
data_mixes.head()

,CSR_carb_Производитель 10,CSR_carb_Производитель 13,CSR_carb_Производитель 14,CSR_carb_Производитель 15,CSR_carb_Производитель 16,CSR_carb_Производитель 17,CSR_carb_Производитель 18,CSR_carb_Производитель 19,CSR_carb_Производитель 2,CSR_carb_Производитель 22,CSR_carb_Производитель 24,CSR_carb_Производитель 29,CSR_carb_Производитель 3,CSR_carb_Производитель 36,CSR_carb_Производитель 37,CSR_carb_Производитель 43,CSR_carb_Производитель 5,CSR_carb_Производитель 6,CSR_carb_Производитель 7,CSR_carb_Производитель 8,I_Производитель 10,I_Производитель 13,I_Производитель 14,I_Производитель 15,I_Производитель 16,I_Производитель 17,I_Производитель 18,I_Производитель 19,I_Производитель 2,I_Производитель 22,I_Производитель 24,I_Производитель 29,I_Производитель 3,I_Производитель 36,I_Производитель 37,I_Производитель 43,I_Производитель 5,I_Производитель 6,I_Производитель 7,I_Производитель 8,Io_Производитель 10,Io_Производитель 13,Io_Производитель 14,Io_Производитель 15,Io_Производитель 16,Io_Производитель 17,Io_Производитель 18,Io_Производитель 19,Io_Производитель 2,Io_Производитель 22,Io_Производитель 24,Io_Производитель 29,Io_Производитель 3,Io_Производитель 36,Io_Производитель 37,Io_Производитель 43,Io_Производитель 5,Io_Производитель 6,Io_Производитель 7,Io_Производитель 8,KTC_Производитель 10,KTC_Производитель 13,KTC_Производитель 14,KTC_Производитель 15,KTC_Производитель 16,KTC_Производитель 17,KTC_Производитель 18,KTC_Производитель 19,KTC_Производитель 2,KTC_Производитель 22,KTC_Производитель 24,KTC_Производитель 29,KTC_Производитель 3,KTC_Производитель 36,KTC_Производитель 37,KTC_Производитель 43,KTC_Производитель 5,KTC_Производитель 6,KTC_Производитель 7,KTC_Производитель 8,MF_otosh_Производитель 10,MF_otosh_Производитель 13,MF_otosh_Производитель 14,MF_otosh_Производитель 15,MF_otosh_Производитель 16,MF_otosh_Производитель 17,MF_otosh_Производитель 18,MF_otosh_Производитель 19,MF_otosh_Производитель 2,MF_otosh_Производитель 22,MF_otosh_Производитель 24,MF_otosh_Производитель 29,MF_otosh_Производитель 3,MF_otosh_Производитель 36,MF_otosh_Производитель 37,MF_otosh_Производитель 43,MF_otosh_Производитель 5,MF_otosh_Производитель 6,MF_otosh_Производитель 7,MF_otosh_Производитель 8,MF_spec_Производитель 10,MF_spec_Производитель 13,MF_spec_Производитель 14,MF_spec_Производитель 15,MF_spec_Производитель 16,MF_spec_Производитель 17,MF_spec_Производитель 18,MF_spec_Производитель 19,MF_spec_Производитель 2,MF_spec_Производитель 22,MF_spec_Производитель 24,MF_spec_Производитель 29,MF_spec_Производитель 3,MF_spec_Производитель 36,MF_spec_Производитель 37,MF_spec_Производитель 43,MF_spec_Производитель 5,MF_spec_Производитель 6,MF_spec_Производитель 7,MF_spec_Производитель 8,MF_Производитель 10,MF_Производитель 13,MF_Производитель 14,MF_Производитель 15,MF_Производитель 16,MF_Производитель 17,MF_Производитель 18,MF_Производитель 19,MF_Производитель 2,MF_Производитель 22,MF_Производитель 24,MF_Производитель 29,MF_Производитель 3,MF_Производитель 36,MF_Производитель 37,MF_Производитель 43,MF_Производитель 5,MF_Производитель 6,MF_Производитель 7,MF_Производитель 8,R0_Производитель 10,R0_Производитель 13,R0_Производитель 14,R0_Производитель 15,R0_Производитель 16,R0_Производитель 17,R0_Производитель 18,R0_Производитель 19,R0_Производитель 2,R0_Производитель 22,R0_Производитель 24,R0_Производитель 29,R0_Производитель 3,R0_Производитель 36,R0_Производитель 37,R0_Производитель 43,R0_Производитель 5,R0_Производитель 6,R0_Производитель 7,R0_Производитель 8,SI_Производитель 10,SI_Производитель 13,SI_Производитель 14,SI_Производитель 15,SI_Производитель 16,SI_Производитель 17,SI_Производитель 18,SI_Производитель 19,SI_Производитель 2,SI_Производитель 22,SI_Производитель 24,SI_Производитель 29,SI_Производитель 3,SI_Производитель 36,SI_Производитель 37,SI_Производитель 43,SI_Производитель 5,SI_Производитель 6,SI_Производитель 7,SI_Производитель 8,Vt_Производитель 10,Vt_Производитель 13,Vt_Производитель 14,Vt_Производитель 15,Vt_Производитель 16,Vt_Производит

In [5]:
def mixes_prop_average(data_mixes):
    
    def added_col(X):
        xxx=0
        for i in range(1,60):
            column = '{}'.format(X)+' {}'.format(i)
            if column in data_mixes.columns:
                xxx+= data_mixes[column]/100*data_mixes[f'Производитель {i}']
                data_mixes.drop(columns=[column],inplace=True)
            else:
                continue
            data_mixes[X+'_Aver']=xxx
    
        return data_mixes
        
    cols = ['R0_Производитель', 'sigma_Производитель', 'Vt_Производитель', 'I_Производитель', 'Io_Производитель', 
        'SI_Производитель', 'KTC_Производитель', 'MF_Производитель',
       'MF_spec_Производитель', 'MF_otosh_Производитель', 'CSR_carb_Производитель']

    for col in cols:
        added_col(col)
    
    return data_mixes

data_mixes = mixes_prop_average(data_mixes)
data_mixes.head()

,year,Дата,"Качество кокса,% (Гранулометрический состав,% — ""+80"")","Качество кокса,% (Гранулометрический состав,% — ""25-0"")","Качество кокса,% (Гранулометрический состав,% — ""40-25"")","Качество кокса,% (Гранулометрический состав,% — ""60-40"")","Качество кокса,% (Гранулометрический состав,% — ""80-60"")","Качество кокса,% (Показатели прочности,% — CRI)","Качество кокса,% (Показатели прочности,% — CSR)","Качество кокса,% (Показатели прочности,% — M10)","Качество кокса,% (Показатели прочности,% — M25)","Качество кокса,% (Технический анализ,% — влага)","Качество кокса,% (Технический анализ,% — зола)","Качество кокса,% (Технический анализ,% — летуч.)","Качество кокса,% (Технический анализ,% — сера)","Качество шихты (Мд. Серы,%)","Качество шихты (Технический анализ, % — влага)","Качество шихты (Технический анализ, % — зола)","Качество шихты (Технический анализ, % — летуч.)",Качество шихты (пласт. слой мм),"Качество шихты (помол,%)","Качество шихты (пыль,%)","Оборот печей, час",Производитель 10,Производитель 11,Производитель 12,Производитель 13,Производитель 14,Производитель 15,Производитель 16,Производитель 17,Производитель 18,Производитель 19,Производитель 2,Производитель 20,Производитель 21,Производитель 22,Производитель 23,Производитель 24,Производитель 25,Производитель 26,Производитель 27,Производитель 28,Производитель 29,Производитель 3,Производитель 30,Производитель 31,Производитель 32,Производитель 33,Производитель 34,Производитель 35,Производитель 36,Производитель 37,Производитель 38,Производитель 39,Производитель 4,Производитель 40,Производитель 41,Производитель 42,Производитель 43,Производитель 44,Производитель 45,Производитель 46,Производитель 47,Производитель 48,Производитель 49,Производитель 5,Производитель 50,Производитель 51,Производитель 52,Производитель 53,Производитель 54,Производитель 55,Производитель 56,Производитель 57,Производитель 58,Производитель 6,Производитель 7,Производитель 8,Производитель 9,Расчетн расх. Коэф угл/кокс т/т,"Состав шихты,%(в т.ч. Марка Ж)","Состав шихты,%(в т.ч. кокс.)","Состав шихты,%(отощ.)","Состав шихты,%(спек.)","Средн. Размер куска кокса,мм",R0_Производитель_Aver,sigma_Производитель_Aver,Vt_Производитель_Aver,I_Производитель_Aver,Io_Производитель_Aver,SI_Производитель_Aver,KTC_Производитель_Aver,MF_Производитель_Aver,MF_spec_Производитель_Aver,MF_otosh_Производитель_Aver,CSR_carb_Производитель_Aver
0,2017,2017-01-01,14.5,3.2,9.4,32.8,40.1,NaN,NaN,7.8,88.3,NaN,11.3,1.1,0.61,0.63,8.4,8.8,26.1,17.0,77.9,39.9,19.5,10.0,6.0,0.0,0.0,12.0,0.0,0.0,20.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.324987,29.8,28.0,50.0,50.0,60.9750,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
1,2017,2017-01-02,5.3,2.9,13.1,44.0,34.7,28.4,60.9,7.4,88.5,NaN,11.0,1.0,0.59,0.62,8.5,8.5,25.2,16.0,77.0,40.1,19.5,10.0,6.0,0.0,0.0,12.0,0.0,0.0,20.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.315316,29.8,28.0,50.0,50.0,55.6800,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
2,2017,2017-01-03,11.2,2.4,6.1,32.8,47.5,NaN,NaN,7.7,88.3,NaN,10.9,1.2,0.59,0.63,9.2,8.1,25.7,15.0,76.6,39.1,19.5,10.0,6.0,0.0,0.0,12.0,0.0,0.0,20.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.320671,29.8,28.0,50.0,50.0,62.0125,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
3,2017,2017-01-04,20.3,2.8,6.2,25.9,44.8,NaN,NaN,7.7,88.2,NaN,11.3,1.2,0.60,0.63,9.2,8.2,25.7,15.0,76.1,38.7,19.5,10.0,6.0,0.0,0.0,12.0,0.0,0.0,20.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,

In [6]:
def target_creat(dataframe):

    columns = ['Дата',
               'Качество кокса,% (Показатели прочности,% — CSR)',
               'Качество кокса,% (Показатели прочности,% — M10)',
               'Качество кокса,% (Показатели прочности,% — M25)']
    target = pd.DataFrame(dataframe, columns=columns)

    #rename columns
    target.rename(columns={'Дата':'ДатаПробыКокса', 
                            'Качество кокса,% (Показатели прочности,% — CSR)': 'CSR',
                            'Качество кокса,% (Показатели прочности,% — M10)':'M10',
                            'Качество кокса,% (Показатели прочности,% — M25)':'M25'}, inplace=True)
    # new column
    target["ДатаШихтовки"] = target["ДатаПробыКокса"] - pd.Timedelta(4,unit="d")
    
    # recording
    target.to_csv("target_b1.csv")
    
    return target
    
target = target_creat(data_mixes)
target.head()

,ДатаПробыКокса,CSR,M10,M25,ДатаШихтовки
0,2017-01-01,NaN,7.8,88.3,2016-12-28
1,2017-01-02,60.9,7.4,88.5,2016-12-29
2,2017-01-03,NaN,7.7,88.3,2016-12-30
3,2017-01-04,NaN,7.7,88.2,2016-12-31
4,2017-01-05,NaN,7.7,88.7,2017-01-01


In [7]:
def feature_creat(dataframe):

    mix_cols = {
    "Дата": "ДатаШихтовки",
    "Состав шихты,%(спек.)": "% спек",
    "Состав шихты,%(в т.ч. Марка Ж)": "% марка Ж",
    "Состав шихты,%(в т.ч. кокс.)": "% кокс.",
    "Оборот печей, час": "Оборот печей, ч",
    "Качество шихты (помол,%)": "Помол, %",
    "Качество шихты (пыль,%)": "Пыль, %",
    "Качество шихты (Технический анализ, % — влага)": "Влага, %",
    "Качество шихты (Технический анализ, % — зола)": "Зола, %",
    "Качество шихты (Технический анализ, % — летуч.)": "Летуч., %",
    "Качество шихты (Мд. Серы,%)": "Мд. Серы, %",
    "Качество шихты (пласт. слой мм)": "Пласт. слой, мм"}
    
    prop_cols=['R0_Производитель_Aver','sigma_Производитель_Aver',
          'Vt_Производитель_Aver','I_Производитель_Aver',
          'Io_Производитель_Aver', 'SI_Производитель_Aver','KTC_Производитель_Aver',
          'MF_Производитель_Aver','MF_spec_Производитель_Aver',
          'MF_otosh_Производитель_Aver','CSR_carb_Производитель_Aver']
    
    feature = pd.DataFrame(data_mixes,columns=list(mix_cols.keys())+prop_cols)
    
    #rename columns
    feature.rename(columns=mix_cols, inplace=True)
    
    # recording
    feature.to_csv("feature_b1.csv")
    
    return feature
    
feature = feature_creat(data_mixes)
feature.head()

,ДатаШихтовки,% спек,% марка Ж,% кокс.,"Оборот печей, ч","Помол, %","Пыль, %","Влага, %","Зола, %","Летуч., %","Мд. Серы, %","Пласт. слой, мм",R0_Производитель_Aver,sigma_Производитель_Aver,Vt_Производитель_Aver,I_Производитель_Aver,Io_Производитель_Aver,SI_Производитель_Aver,KTC_Производитель_Aver,MF_Производитель_Aver,MF_spec_Производитель_Aver,MF_otosh_Производитель_Aver,CSR_carb_Производитель_Aver
0,2017-01-01,50.0,29.8,28.0,19.5,77.9,39.9,8.4,8.8,26.1,0.63,17.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
1,2017-01-02,50.0,29.8,28.0,19.5,77.0,40.1,8.5,8.5,25.2,0.62,16.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
2,2017-01-03,50.0,29.8,28.0,19.5,76.6,39.1,9.2,8.1,25.7,0.63,15.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
3,2017-01-04,50.0,29.8,28.0,19.5,76.1,38.7,9.2,8.2,25.7,0.63,15.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66
4,2017-01-05,50.0,29.8,28.0,19.5,77.6,39.0,8.2,8.6,25.1,0.62,17.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66


In [8]:
def aligned_creat(feature,target):
    aligned_data = pd.merge(feature, target, on=["ДатаШихтовки"])
    aligned_data.to_csv("aligned_data_b1.csv")
    
    #add columns
    aligned_data["Оборот печей отсеч, ч"] = np.where(aligned_data['Оборот печей, ч'] >= 18,
                                                        18, aligned_data['Оборот печей, ч'])
    
    # recording
    aligned_data.to_csv("aligned_data_b1.csv")
    
    return aligned_data

aligned_data = aligned_creat(feature,target)
aligned_data.head(5)

,ДатаШихтовки,% спек,% марка Ж,% кокс.,"Оборот печей, ч","Помол, %","Пыль, %","Влага, %","Зола, %","Летуч., %","Мд. Серы, %","Пласт. слой, мм",R0_Производитель_Aver,sigma_Производитель_Aver,Vt_Производитель_Aver,I_Производитель_Aver,Io_Производитель_Aver,SI_Производитель_Aver,KTC_Производитель_Aver,MF_Производитель_Aver,MF_spec_Производитель_Aver,MF_otosh_Производитель_Aver,CSR_carb_Производитель_Aver,ДатаПробыКокса,CSR,M10,M25,"Оборот печей отсеч, ч"
0,2017-01-01,50.0,29.8,28.0,19.5,77.9,39.9,8.4,8.8,26.1,0.63,17.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66,2017-01-05,NaN,7.7,88.7,18.0
1,2017-01-02,50.0,29.8,28.0,19.5,77.0,40.1,8.5,8.5,25.2,0.62,16.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66,2017-01-06,NaN,7.5,88.7,18.0
2,2017-01-03,50.0,29.8,28.0,19.5,76.6,39.1,9.2,8.1,25.7,0.63,15.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66,2017-01-07,NaN,7.6,88.6,18.0
3,2017-01-04,50.0,29.8,28.0,19.5,76.1,38.7,9.2,8.2,25.7,0.63,15.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66,2017-01-08,NaN,7.6,88.1,18.0
4,2017-01-05,50.0,29.8,28.0,19.5,77.6,39.0,8.2,8.6,25.1,0.62,17.0,1.016755,0.059921,62.14549,21.998627,0.20525,5.81,0.7408,14116.4,14095.0,21.4,41.66,2017-01-09,60.7,7.8,88.5,18.0
